### Forecasting of electricity consumption using a simple neural network

In [34]:
import torch
import pandas as pd
import numpy as np

In [35]:
# Load the data
data = pd.read_csv('electricity_demand.csv', header=0, index_col=0)

In [36]:
# Prepare the data
# Split the data into training and testing sets
train_size = int(len(data) * 0.8)
train_data = data.iloc[:train_size]
test_data = data.iloc[train_size:]

In [37]:
# Convert the data into PyTorch tensors
train_inputs = torch.from_numpy(train_data.values[:-1, :])
train_targets = torch.from_numpy(train_data.values[1:, :])
test_inputs = torch.from_numpy(test_data.values[:-1, :])
test_targets = torch.from_numpy(test_data.values[1:, :])

In [38]:
# Define the neural network model
class ElectricityDemandForecasting(torch.nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super().__init__()
        self.hidden_layer1 = torch.nn.Linear(input_size, hidden_size)
        self.hidden_layer2 = torch.nn.Linear(hidden_size, hidden_size)
        self.output_layer = torch.nn.Linear(hidden_size, output_size)

    def forward(self, input):
        hidden = torch.relu(self.hidden_layer1(input))
        hidden = torch.relu(self.hidden_layer2(hidden))
        output = self.output_layer(hidden)
        return output

In [39]:
# Initialize the model and optimizer
model = ElectricityDemandForecasting(input_size=1, hidden_size=20, output_size=1)
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

In [40]:
# Train the model
for epoch in range(100):
    # Forward pass
    train_predictions = model(train_inputs.float())
    train_loss = torch.nn.MSELoss()(train_predictions, train_targets.float())

    # Backward pass
    optimizer.zero_grad()
    train_loss.backward()
    optimizer.step()

    # Print the training loss every 10 epochs
    if (epoch + 1) % 10 == 0:
        print(f'Epoch {epoch + 1}, Train Loss: {train_loss.item()}')

Epoch 10, Train Loss: 0.1445731222629547
Epoch 20, Train Loss: 0.09090759605169296
Epoch 30, Train Loss: 0.05719895288348198
Epoch 40, Train Loss: 0.0410848893225193
Epoch 50, Train Loss: 0.03599340096116066
Epoch 60, Train Loss: 0.03366927430033684
Epoch 70, Train Loss: 0.030817747116088867
Epoch 80, Train Loss: 0.027842313051223755
Epoch 90, Train Loss: 0.024995854124426842
Epoch 100, Train Loss: 0.022139325737953186


In [41]:
# Evaluate the model on the test set
test_predictions = model(test_inputs.float())
test_loss = torch.nn.MSELoss()(test_predictions, test_targets.float())
print(f'Test Loss: {test_loss.item()}')

Test Loss: 0.012079893611371517


In [42]:
# Save the trained model
torch.save(model.state_dict(), "model.pth")

In [43]:
# # Load the saved model
# model.load_state_dict(torch.load("model.pth"))
# model.eval()

# # Predict demand for new input
# with torch.no_grad():
#     predicted_demand = model(new_input)


In [44]:
# Convert the test predictions and test output to NumPy arrays
test_predictions = test_predictions.detach().numpy()


In [45]:
test_predictions_df = pd.DataFrame(test_predictions, columns=['Predicted Demand'])
test_predictions_df.to_csv('test_predictions.csv', index=False)

In [46]:
# # import pandas as pd
# import altair as alt

# # # Convert the test predictions and test output to Pandas dataframes
# # test_predictions_df = pd.DataFrame(test_predictions, columns=['Predicted Demand'])
# # test_output_df = pd.DataFrame(test_output, columns=['Actual Demand'])

# # Concatenate the two dataframes
# results_df = pd.concat([test_targets_df, test_predictions_df], axis=0)

# # Create an Altair chart
# chart = alt.Chart(results_df).mark_line().encode(
#     # x='index:Q',
#     y=alt.Y('value:Q', axis=alt.Axis(title='Electricity Demand (kW)')),
#     color='variable:N'
# ).properties(
#     title='Electricity Demand Forecasting Results',
#     width=600,
#     height=400
# ).transform_fold(
#     fold=['Actual Demand', 'Predicted Demand'],
#     as_=['variable', 'value']
# ).interactive()

# # Display the chart
# chart